In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import PchipInterpolator
from scipy.stats import norm
from statsmodels.distributions.empirical_distribution import ECDF

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# MSE imports
import kepler_sieve
from asteroid_element import load_ast_elt
from candidate_element import perturb_elts
from nearest_element import nearest_ast_elt, plot_elt_transform_pdf, plot_elt_transform_map
from nearest_element import ast_elt_transform, calc_beta, make_interp_x

In [2]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

In [3]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [4]:
# Set number of sample points
N_samp_u: int = 2**16
z_range: float = 6.0
N_samp_z: int = int(200*z_range + 1)

# Sample CDF levels: N_samp evenly spaced
cdf_samp_u = (np.arange(N_samp_u) + 0.5) / N_samp_u

# Sample CDF levels: z_samp points evenly spaced by Z
z_samp = np.linspace(-z_range, z_range, N_samp_z)
cdf_samp_z = norm.cdf(z_samp)
pdf_samp_z = norm.pdf(z_samp)

# Combine the two sets of sample points
cdf_samp = np.unique(np.hstack([cdf_samp_u, cdf_samp_z]))

In [5]:
# Review orbital elements
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255499,1255499,2019 QG,58600.0,0.822197,0.237862,0.220677,5.066979,3.770460,0.503214,21.55,0.15,JPL 1,0.807024,272.309165,0.023074,-3.225717,-2.921908,-3.728932,-21.808984
1255501,1255501,2019 QL,58600.0,2.722045,0.530676,0.113833,4.741919,2.351059,5.297173,19.21,0.15,JPL 1,-2.082964,1640.368337,0.003830,-0.176219,-1.273172,0.809793,257.420824
1255502,1255502,2019 QQ,58600.0,1.053137,0.389091,0.172121,5.648270,2.028352,3.266522,25.31,0.15,JPL 1,-3.081905,394.753268,0.015917,-1.623227,-1.688469,1.393436,189.527723


## Build and Plot Transformed Orbital Elements

In [6]:
# Build transformed elements and interpolators
ast_elt_xf, interp_tbl = ast_elt_transform(ast_elt)

In [7]:
# Review orbital elements with extra transformed columns
ast_elt_xf

,Num,Name,a,e,inc,Omega,omega,f,epoch_mjd,log_a,...,cos_f,log_a_z,e_z,sin_inc_z,sin_Omega_z,cos_Omega_z,sin_omega_z,cos_omega_z,sin_f_z,cos_f_z
Num,,,,,,,,,,,,,,,,,,,,,
1,1,Ceres,2.769165,0.076009,0.184901,1.401596,1.284522,1.501306,58600.0,1.018546,...,0.069434,0.371682,-0.920343,0.481007,1.561032,0.136310,1.337657,0.227204,2.034043,0.290861
2,2,Pallas,2.772466,0.230337,0.608007,3.020817,5.411373,1.490912,58600.0,1.019737,...,0.079800,0.382073,1.066688,2.849431,-0.053389,-1.727551,-0.759020,0.586498,1.975070,0.299035
3,3,Juno,2.669150,0.256942,0.226699,2.964490,4.330836,0.996719,58600.0,0.981760,...,0.543060,0.080554,1.340379,0.811157,-0.004199,-1.540834,-1.164045,-0.312397,0.940136,0.691813
4,4,Vesta,2.361418,0.088721,0.124647,1.811840,2.630709,-4.436417,58600.0,0.859262,...,-0.272482,-0.791466,-0.715146,0.010545,1.374974,-0.162143,0.427263,-0.982996,1.388823,0.013821
5,5,Astraea,2.574249,0.191095,0.093672,2.470978,6.260280,-1.738676,58600.0,0.945558,...,-0.167092,-0.257131,0.591323,-0.347369,0.453227,-0.740831,-0.010034,2.448863,-1.623092,0.101835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255499,1255499,2019 QG,0.822197,0.237862,0.220677,5.066979,3.770460,0.807024,58600.0,-0.195775,...,0.691651,-3.391558,1.148620,0.761861,-1.393330,0.277830,-0.520644,-0.841008,0.730276,0.856645
1255501,1255501,2019 QL,2.722045,0.530676,0.113833,4.741919,2.351059,-2.082964,58600.0,1.001383,...,-0.490068,0.233559,2.445225,-0.099482,-2.464143,0.034561,0.687151,-0.670917,-0.995557,-0.188852
1255502,1255502,2019 QQ,1.053137,0.389091,0.172121,5.648270,2.028352,-3.081905,58600.0,0.051773,...,-0.998219,-2.821553,2.087920,0.375099,-0.717939,0.813327,1.060234,-0.376565,-0.038917,-1.949413


## Semimajor Axis, a

In [ ]:
# Plot PDF of a
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='log_a')

In [ ]:
# Plot transform of a
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='log_a')

## Eccentricity, e

In [ ]:
# Plot PDF of e
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='e')

In [ ]:
# Plot transform of e
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='e')

## Inclination, inc

In [ ]:
# Plot PDF of sin_inc
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_inc')

In [ ]:
# Plot transform of sin_inc
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_inc')

## Omega

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_Omega')

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_Omega')

In [ ]:
# Plot PDF of cos_Omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_Omega')

In [ ]:
# Plot PDF of cos_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='cos_Omega')

## omega

In [ ]:
# Plot PDF of sin_omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_omega')

In [ ]:
# Plot map of sin_omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_omega')

In [ ]:
# Plot PDF of cos_omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_omega')

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='cos_omega')

## True Anomaly, f

In [ ]:
# Plot PDF of sin_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_f')

In [ ]:
# Plot map of sin_omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_f')

In [ ]:
# Plot PDF of cos_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_f')

In [ ]:
# Plot PDF of cos_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_f')

## Covariance of Transformed Elements

In [ ]:
# Build tranformation matrix beta and X_beta for computing distance to orbital elements
beta, X_beta = calc_beta(ast_elt_xf)

In [ ]:
# Assemble covaraiance matrix
Q = np.cov(X_beta, rowvar=False)

In [ ]:
# Demonstrate that Q is the 9x9 identity matrix
Q_err = np.linalg.norm(Q - np.identity(9))
print(f'Error of matrix Q vs. identity: {Q_err:6.2e}')

## Search for Asteroid with Nearest Orbital Elements

In [8]:
# First 32 odd numbered asteroids (don't want consecutive to disambiguate rows and columns)
elt = ast_elt.iloc[0:64:2].copy()

In [9]:
# Search for near elements on unperturbed asteroids
ast_elt_near = nearest_ast_elt(elt)

In [11]:
# Review near elements - should be exact matches with 0 distance
ast_elt_near.iloc[0:10]

,Num,Name,Q_norm,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,,
1,1,Ceres,0.0,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
3,3,Juno,0.0,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
5,5,Astraea,0.0,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
7,7,Iris,0.0,58600.0,2.385334,0.231206,0.096406,4.530233,2.535354,2.450785,5.51,0.15,JPL 109,-3.593901,1345.619184,0.004669,3.233187,-2.811498,0.782402,-524.864951
9,9,Metis,0.0,58600.0,2.385637,0.123114,0.097334,1.202682,0.112004,4.832147,6.28,0.17,JPL 114,-1.697553,1345.875332,0.004668,-0.136353,-0.382867,1.314686,310.816364
11,11,Parthenope,0.0,58600.0,2.453109,0.100472,0.080807,2.191201,3.412998,4.868259,6.55,0.15,JPL 103,4.667173,1403.375214,0.004477,-2.093912,3.988187,-0.678986,316.029636
13,13,Egeria,0.0,58600.0,2.575981,0.085121,0.288610,0.754365,1.405772,3.272292,6.74,0.15,JPL 87,-3.030955,1510.123393,0.004161,-0.850756,-0.870818,2.160137,723.649002
15,15,Eunomia,0.0,58600.0,2.644100,0.186084,0.205119,5.112669,1.719126,4.946048,5.28,0.23,JPL 74,-1.710809,1570.418183,0.004001,-0.788527,-1.162199,0.548610,334.203757
17,17,Thetis,0.0,58600.0,2.470354,0.133032,0.097585,2.191312,2.377282,5.294707,7.76,0.15,JPL 106,-1.230547,1418.199216,0.004430,3.580116,3.338047,4.568595,223.112824


In [17]:
# Inputs to perturb elements: large
sigma_a = 0.01
sigma_e = 0.002
sigma_inc_deg = 0.25
sigma_f_deg = 0.5
sigma_Omega_deg = 0.5
sigma_omega_deg = 0.5
mask_pert = None
random_seed = 42

# Perturb orbital elements
elts_pert = perturb_elts(elt, sigma_a=sigma_a, sigma_e=sigma_e, 
                    sigma_inc_deg=sigma_inc_deg, sigma_f_deg=sigma_f_deg, 
                    sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                    mask_pert=mask_pert, random_seed=random_seed)

In [18]:
# Search for near elements on perturbed asteroids
ast_elt_near_pert = nearest_ast_elt(elts_pert)

In [19]:
# Review near elements - should be close but not zero
ast_elt_near_pert

,Num,Name,Q_norm,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,,
1,1,Ceres,0.054378,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
3,3,Juno,0.055507,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
5,5,Astraea,0.221987,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
7,7,Iris,0.142874,58600.0,2.385334,0.231206,0.096406,4.530233,2.535354,2.450785,5.51,0.15,JPL 109,-3.593901,1345.619184,0.004669,3.233187,-2.811498,0.782402,-524.864951
9,9,Metis,0.068824,58600.0,2.385637,0.123114,0.097334,1.202682,0.112004,4.832147,6.28,0.17,JPL 114,-1.697553,1345.875332,0.004668,-0.136353,-0.382867,1.314686,310.816364
11,11,Parthenope,0.051615,58600.0,2.453109,0.100472,0.080807,2.191201,3.412998,4.868259,6.55,0.15,JPL 103,4.667173,1403.375214,0.004477,-2.093912,3.988187,-0.678986,316.029636
13,13,Egeria,0.157083,58600.0,2.575981,0.085121,0.288610,0.754365,1.405772,3.272292,6.74,0.15,JPL 87,-3.030955,1510.123393,0.004161,-0.850756,-0.870818,2.160137,723.649002
15,15,Eunomia,0.100659,58600.0,2.644100,0.186084,0.205119,5.112669,1.719126,4.946048,5.28,0.23,JPL 74,-1.710809,1570.418183,0.004001,-0.788527,-1.162199,0.548610,334.203757
17,17,Thetis,0.020899,58600.0,2.470354,0.133032,0.097585,2.191312,2.377282,5.294707,7.76,0.15,JPL 106,-1.230547,1418.199216,0.004430,3.580116,3.338047,4.568595,223.112824
